## Classification model visualization

In [23]:
# package version 설정 
!pip install --user torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0 torchtext==0.10.0

In [2]:
# fastai pytorch 자동화 라이브러리 
from fastai.vision.all import *

In [3]:
# path 설정 
root_path = Path("../input")
train_csv_path = root_path/'happy-whale-and-dolphin/train.csv'
train_img_path = root_path/"jpeg-happywhale-128x128/train_images-128-128/train_images-128-128"

## Get a glimpse of the files
`get_image_files` 를 통해 경로 내의 모든 파일 load

In [4]:
train_imgs = get_image_files(train_img_path)
train_imgs[:10]

In [5]:
Image.open(train_imgs[1])

In [6]:
train_path_df = pd.read_csv(train_csv_path)

In [7]:
train_path_df.head(5)

In [24]:
# csv 파일 내의 경로와 이미지 경로 merge
train_path_df['image'] = train_path_df['image'].apply(lambda x:train_img_path/x)

In [25]:
train_path_df.head(5)

In [10]:
Image.open(train_path_df.image[0])

## Datablock
- fastai 내의 `DataBlock` class 사용

In [28]:
ColReader(0)

In [27]:
ColReader(1)

In [11]:
# https://docs.fast.ai/data.block.html
data = DataBlock(blocks = (ImageBlock, CategoryBlock),
                splitter = TrainTestSplitter(),
                 get_x = ColReader(0),
                 get_y = ColReader(1),
                 item_tfms = Resize(224),
                 batch_tfms = aug_transforms(size = 128))

* `ImageBlock`: image data
* `CategoryBlock`: targets
* `get_x` ,`get_y`: the image path from column 0 of dataframe and the target from column 1 of the dataframe
* `splitter`: train and test data are split

## Dataloader

`train_path_df` 

`bs`: batch size

In [12]:
dls = data.dataloaders(train_path_df, bs = 128)

In [13]:
dls.show_batch()

## Model

## Creating a simple CNN learner
`cnn_learner`: convolutional neural network, (dataloader, a model architecture, loss function)

`models.resnet18`: pretrained model 

In [14]:
learn = cnn_learner(dls, models.resnet18, loss_func = CrossEntropyLossFlat(), ps = 0.25)

`fine_tune`: Learner.fine_tune(epochs, base_lr=0.002, freeze_epochs=1, lr_mult=100, pct_start=0.3, div=5.0, lr_max=None, div_final=100000.0, wd=None, moms=None, cbs=None, reset_opt=False)

In [15]:
learn.fine_tune(2, 3e-3)

예측 결과, 실제 결과 확인

In [16]:
learn.show_results()

## CAM 기반 결과 해석 

참고: CAM [fastbook](https://github.com/fastai/fastbook/blob/master/18_CAM.ipynb) 

In [18]:
Image.open(train_path_df.image[6])

Let's grab a batch.

In [19]:
img = PILImage.create(train_path_df.image[6])
# grab a batch
x, = first(dls.test_dl([img]))

Hook class 정의 

model, input, output 필요

In [20]:
class Hook():
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_func)   
    def hook_func(self, m, i, o): self.stored = o.detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()

activation과 weight matrix 간의 dot product 연산 수행 

act 변수에 마지막 layer에 대한 activation 저장 

`torch.einsum`을 통해 weight와 activation 간 dot product 실시 

In [21]:
with Hook(learn.model[0]) as hook:
    with torch.no_grad(): 
        output = learn.model.eval()(x.cuda())
        act = hook.stored[0]
    cam_map = torch.einsum('ck,kij->cij', learn.model[1][-1].weight, act)
    cam_map.shape

In [22]:
# 7x7 feature map을 통한 cam map visualization
x_dec = TensorImage(dls.train.decode((x,))[0][0])
_,ax = plt.subplots()
x_dec.show(ctx=ax)
ax.imshow(cam_map[1].detach().cpu(), alpha=0.6, extent=(0, 128, 128, 0),
              interpolation='bilinear', cmap='magma');